In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
import os
import time
import pandas as pd
from os import listdir
from os.path import join
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [3]:
def extract_time_domain_features(df):
    """
    Розраховує часові ознаки для даних акселерометра.

    Параметри:
    - df: DataFrame з даними акселерометра, де стовпці мають назви accelerometer_X, accelerometer_Y, accelerometer_Z.

    Повертає:
    - time_features: DataFrame з розрахованими часовими ознаками.
    """
    time_features = pd.DataFrame()

    # Додає середнє значення для кожного каналу акселерометра
    time_features['mean_x'] = df['accelerometer_X'].mean()
    time_features['mean_y'] = df['accelerometer_Y'].mean()
    time_features['mean_z'] = df['accelerometer_Z'].mean()

    # Додає дисперсію для кожного каналу акселерометра
    time_features['variance_x'] = df['accelerometer_X'].var()
    time_features['variance_y'] = df['accelerometer_Y'].var()
    time_features['variance_z'] = df['accelerometer_Z'].var()

    # Додає медіану для кожного каналу акселерометра
    time_features['median_x'] = df['accelerometer_X'].median()
    time_features['median_y'] = df['accelerometer_Y'].median()
    time_features['median_z'] = df['accelerometer_Z'].median()

    # Додає ковзне середнє для кожного каналу акселерометра
    window_size = 10
    time_features['rolling_mean_x'] = df['accelerometer_X'].rolling(
        window=window_size).mean()
    time_features['rolling_mean_y'] = df['accelerometer_Y'].rolling(
        window=window_size).mean()
    time_features['rolling_mean_z'] = df['accelerometer_Z'].rolling(
        window=window_size).mean()

    return time_features

In [4]:
data_folder = '/content/gdrive/MyDrive/Data_science/hw_05/data'

data = []
labels = []

for activity in ['walking', 'running', 'idle', 'stairs']:
    activity_folder = os.path.join(data_folder, activity)

    for file_name in os.listdir(activity_folder):
        if file_name.endswith('.csv'):
            file_path = os.path.join(activity_folder, file_name)

            df = pd.read_csv(file_path)

            label = activity

            features = extract_time_domain_features(df)

            data.append(features.values.flatten())

            labels.append(label)

In [5]:
df_features = pd.DataFrame(data)
df_labels = pd.Series(labels, name='activity')

X_train, X_test, y_train, y_test = train_test_split(df_features, df_labels, test_size=0.2, random_state=42)

In [23]:
df_features

,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.268630,-8.448180,1.275152
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.114494,-12.301887,-3.526659
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.135925,-9.733867,0.659842
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.817975,-7.799352,-0.106302
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.290772,-10.254366,-0.912670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.621172,-9.106107,-1.039562
6458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.541568,-7.117483,-2.108334
6459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.282516,-8.415140,-2.130361
6460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.237505,-7.107906,-2.465549


In [22]:
df_labels

0       walking
1       walking
2       walking
3       walking
4       walking
         ...   
6457     stairs
6458     stairs
6459     stairs
6460     stairs
6461     stairs
Name: activity, Length: 6462, dtype: object

In [24]:
# Створення об'єкту імпутера
imputer = SimpleImputer(strategy='mean')

In [25]:
# Навчання імпутера на навчальних даних та застосування його до X_train і X_test
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

In [26]:
# SVM
svm_model = SVC()
svm_model.fit(X_train_imputed, y_train)

SVC()

In [27]:
# Випадковий ліс
rf_model = RandomForestClassifier()
rf_model.fit(X_train_imputed, y_train)

RandomForestClassifier()

In [28]:
# Оцінка моделі SVM
start_time = time.time()
svm_predictions = svm_model.predict(X_test_imputed)
svm_accuracy = accuracy_score(y_test, svm_predictions)
svm_report = classification_report(y_test, svm_predictions)
svm_confusion = confusion_matrix(y_test, svm_predictions)
svm_elapsed_time = time.time() - start_time

In [29]:
# Оцінка моделі випадкового лісу
start_time = time.time()
rf_predictions = rf_model.predict(X_test_imputed)
rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_report = classification_report(y_test, rf_predictions)
rf_confusion = confusion_matrix(y_test, rf_predictions)
rf_elapsed_time = time.time() - start_time

In [30]:
# Вивід результатів
print("Результати для SVM:")
print(f"Точність: {svm_accuracy}")
print(f"Звіт про класифікацію:\n{svm_report}")
print(f"Матриця плутанини:\n{svm_confusion}")
print(f"Час виконання SVM: {svm_elapsed_time} сек.")

print("\nРезультати для випадкового лісу:")
print(f"Точність: {rf_accuracy}")
print(f"Звіт про класифікацію:\n{rf_report}")
print(f"Матриця плутанини:\n{rf_confusion}")
print(f"Час виконання випадкового лісу: {rf_elapsed_time} сек.")

Результати для SVM:
Точність: 0.9814385150812065
Звіт про класифікацію:
              precision    recall  f1-score   support

        idle       0.99      1.00      1.00       191
     running       1.00      1.00      1.00       688
      stairs       0.50      0.04      0.08        23
     walking       0.95      1.00      0.97       391

    accuracy                           0.98      1293
   macro avg       0.86      0.76      0.76      1293
weighted avg       0.97      0.98      0.97      1293

Матриця плутанини:
[[191   0   0   0]
 [  1 687   0   0]
 [  0   0   1  22]
 [  0   0   1 390]]
Час виконання SVM: 0.20322489738464355 сек.

Результати для випадкового лісу:
Точність: 0.982985305491106
Звіт про класифікацію:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00       191
     running       1.00      1.00      1.00       688
      stairs       0.56      0.22      0.31        23
     walking       0.96      0.99      0.97       3

### Результати для моделі SVM та випадкового лісу показують високу точність класифікації для більшості класів, таких як "idle", "running" і "walking". Але є деякі важкості у класифікації класу "stairs".

### Для моделі SVM:

Клас "idle" має 99% точності.
Клас "running" 100% точності.
Клас "stairs" має низьку точність, всього 50% точності, і низький recall - 4%. Це може бути пов'язано з обмеженим обсягом даних для класу "stairs".
Клас "walking" має 95% точності.

### Для моделі випадкового лісу:

Клас "idle" та "running" мають 100% точності.
Клас "stairs" має точність 56% і recall 22%. Також є покращення в порівнянні з SVM, але все ще не ідеальне.
Клас "walking" має 96% точності.
Загально, обидві моделі досягли дуже добрих результатів для класів "idle", "running" і "walking". Проте, клас "stairs" є більш викликаючим для класифікації, і його точність може бути покращена за рахунок більшого обсягу даних або використання інших методів.


### Слід зазначити час виконання:

SVM: 0.20322489738464355 сек.

Випадковий ліс: 0.06655168533325195 сек.